# **Creating a basic image classifer**

implementing an image classifer on Fashion-MNIST, a
popular alternative to mnist

In [ ]:
!pip install git+https://github.com/tensorflow/docs

  Cloning https://github.com/tensorflow/docs to /tmp/pip-req-build-25zhwzv9
  Running command git clone --filter=blob:none --quiet https://github.com/tensorflow/docs /tmp/pip-req-build-25zhwzv9
  Resolved https://github.com/tensorflow/docs to commit e21d085d5ed82504ffcec11aa82ebc78f1f2302e
  Preparing metadata (setup.py) ... done
  Created wheel for tensorflow-docs: filename=tensorflow_docs-2025.3.6.10029-py3-none-any.whl size=186351 sha256=cbcd0a9b35310a7ce32f1db97fadf4bb4da6617bfcc0bc2aade86fe2b524cb50
  Stored in directory: /tmp/pip-ephem-wheel-cache-5p9_xz0t/wheels/3e/88/34/48d2789bc9d37b33ddce06bccc454fae0285e5396d0a5be9d9
Successfully built tensorflow-docs


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import tensorflow_docs as tfdocs
import tensorflow_docs.plots # modules that provide utilities and functions to help document TensorFlow models
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer #  A utility to convert categorical labels into binary format (one-hot encoding
from tensorflow.keras import Model
from tensorflow.keras.datasets import fashion_mnist as fm
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D # Neural Network layer
from tensorflow.keras.layers import Dense # Neural Network layer
from tensorflow.keras.layers import Dropout # Neural Network layer
from tensorflow.keras.layers import ELU # Exponential Linear Unit (ELU) activation function to the input
from tensorflow.keras.layers import Flatten # Neural Network layer
from tensorflow.keras.layers import Input # Neural Network layer
from tensorflow.keras.layers import MaxPooling2D # Neural Network layer
from tensorflow.keras.layers import Softmax # Activation functio
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import plot_model

# Defne a function that will load and prepare the dataset. It will normalize the data,
one-hot encode the labels, take a portion of the training set for validation, and wrap
the three data subsets into three separate tf.data.Dataset instances to increase
performance using from_tensor_slices():

In [ ]:
def load_dataset():
    (X_train, y_train), (X_test, y_test) = fm.load_data()
    X_train = X_train.astype('float32') / 255.0
    X_test = X_test.astype('float32') / 255.0
    # Reshape grayscale to include channel dimension.
    X_train = np.expand_dims(X_train, axis=3)
    X_test = np.expand_dims(X_test, axis=3)
    label_binarizer = LabelBinarizer()
    y_train = label_binarizer.fit_transform(y_train)
    y_test = label_binarizer.fit_transform(y_test)
    (X_train, X_val,y_train, y_val) = train_test_split(X_train, y_train,train_size=0.8)
    train_ds = (tf.data.Dataset.from_tensor_slices((X_train,y_train)))
    val_ds = (tf.data.Dataset.from_tensor_slices((X_val, y_val)))
    test_ds = (tf.data.Dataset.from_tensor_slices((X_test, y_test)))

# Implement a function that will build a network similar to LeNet with the addition
of BatchNormalization, which we'll use to make the network faster and most
stable, and Dropout layers, which will help us combat overftting, a situation where
the network loses generalization power due to high variance:

In [ ]:
def build_network():
    input_layer = Input(shape=(28, 28, 1))
    x = Conv2D(filters=20,
    kernel_size=(5, 5),
    padding='same',
    strides=(1, 1))(input_layer)
    x = ELU()(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D(pool_size=(2, 2),
    strides=(2, 2))(x)
    x = Dropout(0.5)(x)

    x = Conv2D(filters=50,
    kernel_size=(5, 5),
    padding='same',
    strides=(1, 1))(x)
    x = ELU()(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D(pool_size=(2, 2),
    strides=(2, 2))(x)
    x = Dropout(0.5)(x)

    x = Flatten()(x)
    x = Dense(units=500)(x)
    x = ELU()(x)
    x = Dropout(0.5)(x)
    x = Dense(10)(x)
    output = Softmax()(x)
    model = Model(inputs=input_layer, outputs=output)
    return model

# Defne a function that takes a model's training history, along with a metric of
interest, to create a plot corresponding to the training and validation of the curves
of such a metric:

In [ ]:
def plot_model_history(model_history, metric, ylim=None):
    plt.style.use('seaborn-darkgrid')
    plotter = tfdocs.plots.HistoryPlotter()
    plotter.plot({'Model': model_history}, metric=metric)
    plt.title(f'{metric.upper()}')
    if ylim is None:
      plt.ylim([0, 1])
    else:
      plt.ylim(ylim)
    plt.savefig(f'{metric}.png')
    plt.close()